In [1]:
%load_ext lab_black

# Necessary installations, imports

In [4]:
import numpy as np
import pandas as pd
import requests
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigs

# Algorithm 1.

That is the algorithm from the main paper of interest. This algorithm is stated to beat SVD-RBF both on sparse and dense nets, when restoring the missing covatiates.

In [ ]:
def CN_VEC(X, A, S, k, return_time=False):
    """
    Args:
        X: array, matrix of all nodes' covariates, rows - objects, cols - covariates (features)
        A: array, Adjacency matrix of the graph (now weighted)
        S: array of indices, nodes, for which we know covariates
        k: number of neighbours
    """
    # mark the starting time
    start_time = time.time()
    # define the sets we are going to re-use later
    all_nodes = set(np.arange(A.shape[0]))
    empty_nodes = all_nodes - set(S)
    X_hat = X.copy()
    C = A.dot(A).A #initially it used to be .A method, which seems to get old
    # I use the .toarray() method instead, as it does quite the same, but is much more readable
    
    # iterate over the empty nodes, managing the logging
    for node_i in tqdm(empty_nodes):
        Ci = np.ones_like(C) * C[node_i]
        Ci[:, node_i] = 0
        np.fill_diagonal(Ci, 0)
        
        Cj = C.copy()
        Cj[:, node_i] = 0
        np.fill_diagonal(Cj, 0)
        
        dists = np.sum((Ci**2 - 2) * (Ci >= 2) + (Cj**2 - 2) * (Cj >= 2) - 2 * Ci * Cj, axis=1)[S]
        
        idxs = np.argsort(dists)[:k]
        ktop = np.array(S)[idxs]
        X_hat[node_i] = np.mean(X[ktop])
        
    # a bit of logging, just to fulfill the curiousity
    print(f'The time, taken to build the X_hat matrix is {time.time() - start_time}')
    if return_time:
        return X_hat, time.time() - start_time
    return X_hat

In [20]:
A = csr_matrix(np.random.randint(0, 10, (3, 3)), dtype=np.float64)
print(A)
print(A.dot(A))
print(A.dot(A).toarray())

  (0, 0)	6.0
  (0, 1)	3.0
  (0, 2)	8.0
  (1, 0)	2.0
  (1, 2)	2.0
  (2, 0)	9.0
  (2, 1)	4.0
  (2, 2)	3.0
  (0, 2)	78.0
  (0, 1)	50.0
  (0, 0)	114.0
  (1, 2)	22.0
  (1, 1)	14.0
  (1, 0)	30.0
  (2, 2)	89.0
  (2, 1)	39.0
  (2, 0)	89.0
[[114.  50.  78.]
 [ 30.  14.  22.]
 [ 89.  39.  89.]]


In [16]:
A.dot(A).A

AttributeError: 'numpy.ndarray' object has no attribute 'A'